In [1]:
import numpy as np


In [6]:
CONTEXT_SIZE = 2


In [11]:
text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis hendrerit ipsum leo, pharetra imperdiet ante auctor eget. Duis fringilla neque a nisi cursus, a consectetur sem vulputate. Nam porttitor gravida nibh, id fringilla leo fringilla non. Mauris mauris odio, rutrum a neque id, accumsan molestie sem. Mauris quis risus in mi tristique efficitur. Nulla feugiat ante magna, vel dignissim justo venenatis vitae. Curabitur felis nisl, auctor at dolor vitae, convallis semper ante. Sed efficitur, felis vel porta lacinia, urna ex pharetra erat, ut tempus nisl elit eu lorem. Sed risus libero, malesuada at mauris finibus, eleifend volutpat lacus. Donec vel nibh leo. Aliquam mattis odio ut convallis fermentum. Nam volutpat dolor dui, et malesuada tellus varius at. Donec pretium enim a nisi egestas, non iaculis purus laoreet. Sed nibh risus, congue vel velit id, varius dapibus lorem. Etiam tincidunt, justo ut pellentesque ultrices, neque odio elementum elit, nec semper quam lacus quis dolor. Suspendisse faucibus ipsum quis metus suscipit porta. Proin vitae vehicula eros, in imperdiet lacus. Ut sed pulvinar mauris.".split()


In [14]:
skipgrams = []
for i in range(CONTEX_SIZE, len(text) - CONTEXT_SIZE):
    array = [text[j
            ] for j in np.arange(i-CONTEXT_SIZE,i+CONTEXT_SIZE+1) if j != i]
    skipgrams.append((text[i],array))

In [15]:
print(skipgrams[0:2])

      

[('dolor', ['Lorem', 'ipsum', 'sit', 'amet,']), ('sit', ['ipsum', 'dolor', 'amet,', 'consectetur'])]
